In [ ]:
%pip install --upgrade --quiet  langchain langchain-community langchain-groq langchain-experimental langchain-google-genai neo4j tiktoken yfiles_jupyter_graphs PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.4 MB/s eta 0:00:00


In [ ]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
from google.colab import userdata
GROQ_API_KEY=userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

In [ ]:
from typing import Tuple, List, Optional

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser

In [ ]:
from langchain_core.runnables import ConfigurableField

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase


In [ ]:
import os

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
from langchain_community.vectorstores import Neo4jVector

In [ ]:
#paste your neo4j uri, username and password here
NEO4J_URI=""
NEO4J_USERNAME=""
NEO4J_PASSWORD=""

In [ ]:
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD

In [ ]:
from langchain_community.graphs import Neo4jGraph

In [ ]:
graph = Neo4jGraph()

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("your/pdf/path")
raw_documents = []
async for page in loader.alazy_load():
    raw_documents.append(page)

In [ ]:
len(raw_documents)

26

In [ ]:
raw_documents[:1]

[Document(metadata={'source': '/content/12. Fine-Tuning Generation Models _ Hands-On Large Language Models.pdf', 'page': 0}, page_content='Chapter 12. Fine-Tuning GenerationModels\nIn this chapter, we will take a pretrained text generation model and go\nover the process of fine-tuning it. This fine-tuning step is key in producing\nhigh-quality models and an important tool in our toolbox to adapt a\nmodel to a specific desired behavior. Fine-tuning allows us to adapt a\nmodel to a specific dataset or domain.\nThroughout this chapter, we will guide you among the two most common\nmethods for fine-tuning text generation models, supervised fine-tuning\nand preference tuning. We will explore the transformative potential of\nfine-tuning pretrained text generation models to make them more effec-\ntive tools for your application.\nThe Three LLM Training Steps:Pretraining, Supervised Fine-Tuning,and Preference Tuning\nThere are three common steps that lead to creating a high-quality LLM:\n1. Lan

In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_documents[:3])

In [ ]:
from langchain_groq import ChatGroq
llm = ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph_documents[:1]

[GraphDocument(nodes=[Node(id='Chapter 12', type='Chapter', properties={}), Node(id='Pretrained Text Generation Model', type='Model', properties={}), Node(id='Fine-Tuning', type='Process', properties={}), Node(id='High-Quality Models', type='Models', properties={}), Node(id='Supervised Fine-Tuning', type='Fine-tuning', properties={}), Node(id='Preference Tuning', type='Fine-tuning', properties={}), Node(id='Language Modeling', type='Step', properties={}), Node(id='Base Models', type='Models', properties={}), Node(id='End User', type='User', properties={}), Node(id='Instructions', type='Concept', properties={}), Node(id='Tokens', type='Unit', properties={}), Node(id='Linguistic And Semantic Representations', type='Representations', properties={}), Node(id='Figure 12-1', type='Figure', properties={})], relationships=[Relationship(source=Node(id='Chapter 12', type='Chapter', properties={}), target=Node(id='Pretrained Text Generation Model', type='Model', properties={}), type='CONTAINS', p

In [ ]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [ ]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

In [ ]:
from yfiles_jupyter_graphs import GraphWidget
from neo4j import GraphDatabase

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

In [ ]:
def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    display(widget)
    return widget

In [ ]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))

GraphWidget(layout=Layout(height='800px', width='100%'))

In [ ]:
from typing import Tuple, List, Optional

In [ ]:
from langchain_community.vectorstores import Neo4jVector
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
from langchain_openai import OpenAIEmbeddings
vector_index = Neo4jVector.from_existing_graph(
    GoogleGenerativeAIEmbeddings(model="models/embedding-001"),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [ ]:
graph.query("CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

[]

In [ ]:
from langchain_core.pydantic_v1 import BaseModel, Field
# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting organization and person entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

In [ ]:
entity_chain = prompt | llm.with_structured_output(Entities)

In [ ]:
entity_chain.invoke({"question": "What is RLHF?"})

Entities(names=['RLHF'])

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [ ]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()


In [ ]:
# Fulltext index query
def structured_retriever(question: str) -> str:
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            """CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        result += "\n".join([el['output'] for el in response])
    return result

In [ ]:
print(structured_retriever("What is RLHF?"))

In [ ]:
def retriever(question: str):
    print(f"Search query: {question}")
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    return final_data

In [ ]:
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

In [ ]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [ ]:
def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [ ]:
_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatGroq(temperature=0, model_name="mixtral-8x7b-32768")
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""

In [ ]:
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke({"question": "What is RLHF"})

Search query: What is RLHF


"RLHF, or Reinforcement Learning with Human Feedback, is a process that aligns a language model's behavior with human expectations. It does this by using human evaluators to score the quality of the model's generations, and then updating the model based on these scores. If the score is high, the model is encouraged to generate more like this type of generation. If the score is low, the model is discouraged from generating such outputs. This process helps to ensure that the model's outputs align with human preferences."

In [ ]:
chain.invoke(
    {
        "question": "What is Training No Reward Model?",
        "chat_history": [("What is RLHF?", "RLHF, or Reinforcement Learning with Human Feedback, is a process that aligns a language model's behavior with human expectations. It does this by using human evaluators to score the quality of the model's generations, and then updating the model based on these scores. If the score is high, the model is encouraged to generate more like this type of generation. If the score is low, the model is discouraged from generating such outputs. This process helps to ensure that the model's outputs align with human preferences.")],
    }
)

Search query: What is the concept of Training a Model without Reward Feedback?


"Training No Reward Model refers to a method of aligning a language model's behavior with human preferences, without using a separate reward model. Instead, it uses a copy of the language model itself as a reference to judge the quality of generations. During training, it calculates the shift in probabilities between the reference and trainable models for accepted and rejected generations, optimizing the likelihood of accepted generations over rejected ones. This method is an alternative to Proximal Policy Optimization (PPO) and can be more cost-effective as it doesn't require training two models."